# Import Package

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

# Load the Data

In [5]:
df = pd.read_excel('Online Retail Data.xlsx', header=0)
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410.0,TEST001,This is a test product.,5.0,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539.0,RETRO SPOTS BUTTER DISH,-1.0,2010-01-04 09:43:00,4.25,14590.0
2,493412.0,TEST001,This is a test product.,5.0,2010-01-04 09:53:00,4.50,12346.0
3,493413.0,21724.0,PANDA AND BUNNIES STICKER SHEET,1.0,2010-01-04 09:54:00,0.85,NaN
4,493413.0,84578.0,ELEPHANT TOY WITH BLUE T-SHIRT,1.0,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991.0,21618.0,4 WILDFLOWER BOTANICAL CANDLES,1.0,2010-12-23 16:49:00,1.25,NaN
461769,539991.0,72741.0,GRAND CHOCOLATECANDLE,4.0,2010-12-23 16:49:00,1.45,NaN
461770,539992.0,21470.0,FLOWER VINE RAFFIA FOOD COVER,1.0,2010-12-23 17:41:00,3.75,NaN
461771,539992.0,22258.0,FELT FARM ANIMAL RABBIT,1.0,2010-12-23 17:41:00,1.25,NaN


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      461773 non-null  object        
 1   product_code  461773 non-null  object        
 2   product_name  459055 non-null  object        
 3   quantity      461773 non-null  float64       
 4   order_date    461773 non-null  datetime64[ns]
 5   price         461773 non-null  float64       
 6   customer_id   360853 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 24.7+ MB


# Data Cleansing

In [8]:
df_clean = df.copy()
# Create a date column
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64')
# Remove all rows without customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# Convert customer_id, product_code, product_name to string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
df_clean['product_code'] = df_clean['product_code'].astype(str)
df_clean['product_name'] = df_clean['product_name'].astype(str)
# Remove all rows without product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# Convert all product_name to lowercase
df_clean['product_name'] = df_clean['product_name'].str.lower()
# Remove all rows with product_code or product_name containing 'test'
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# Delete rows with cancelled status, i.e., those with order_id starting with 'C'.
df_clean = df_clean[df_clean['order_id'].str[:1]!='C']
# Convert negative quantity values to positive since negative values indicate cancelled orders
df_clean['quantity'] = df_clean['quantity'].abs()
# Remove rows with negative price
df_clean = df_clean[df_clean['price']>0]
# Create amount value, which is the multiplication of quantity and price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# Replace product_name from product_code that has multiple product_names with the most frequently occurring one
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# remove outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,amount
0,493414.0,21844.0,red retrospot mug,36.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,91.80
1,493414.0,21533.0,retro spot large milk jug,12.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,51.00
2,493414.0,37508.0,new england ceramic cake server,2.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,5.10
3,493414.0,35001G,hand open shape gold,2.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,8.50
4,493414.0,21527.0,red retrospot traditional teapot,12.0,2010-01-04 10:28:00,6.95,14590.0,2010-01-04,83.40
...,...,...,...,...,...,...,...,...,...
350087,539988.0,84380.0,set of 3 butterfly cookie cutters,1.0,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,1.25
350088,539988.0,84849D,hot baths soap holder,1.0,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,1.69
350089,539988.0,84849B,fairy soap soap holder,1.0,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,1.69
350090,539988.0,22854.0,cream sweetheart egg holder,2.0,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,9.90


In [9]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350092 entries, 0 to 350091
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      350092 non-null  object        
 1   product_code  350092 non-null  object        
 2   product_name  350092 non-null  object        
 3   quantity      350092 non-null  float64       
 4   order_date    350092 non-null  datetime64[ns]
 5   price         350092 non-null  float64       
 6   customer_id   350092 non-null  object        
 7   date          350092 non-null  datetime64[ns]
 8   amount        350092 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(4)
memory usage: 24.0+ MB


# Preparing Basket Data

### Create a Basket DataFrame

In [10]:
basket = pd.pivot_table(df_clean, index='order_id', columns='product_name', values='product_code', aggfunc='nunique', fill_value=0)
basket

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493427.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493428.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493432.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
493433.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539982.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
539985.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
basket.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 16272 entries, 493414.0 to 539988.0
Columns: 3842 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: int64(3842)
memory usage: 477.1 MB


### Encode the basket DataFrame with a value of True for all values above 0 and False for all values equal to 0

In [12]:
def encode(x) :
    if x==0 :
        return False
    if x>0 :
        return True
    
basket_encode = basket.applymap(encode)
basket_encode

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539981.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539985.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Take transactions with more than one unique product

In [13]:
basket_filter = basket_encode [ (basket_encode>0).sum(axis=1)>1]
basket_filter

product_name,10 colour spaceboy pen,12 ass zinc christmas decorations,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 ivory rose peg place settings,12 message cards with envelopes,12 mini toadstool pegs,12 pencil small tube woodland,12 pencils small tube posy,...,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice double planter,zinc heart lattice planter bowl,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc metal heart decoration,zinc police box lantern,zinc top 2 door wooden shelf,zinc willie winkie candle stick
order_id,,,,,,,,,,,,,,,,,,,,,
493414.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493427.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493428.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493432.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
493433.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539978.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539981.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
539982.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
basket_filter.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 15004 entries, 493414.0 to 539988.0
Columns: 3842 entries, 10 colour spaceboy pen to zinc willie winkie  candle stick
dtypes: bool(3842)
memory usage: 55.1 MB


# Applying the Apriori algorithm

### Create a list of frequent itemsets (sets of products frequently purchased)

In [ ]:
# pip install mlxtend
from mlxtend.frequent_patterns import apriori

frequent_itemset = apriori(basket_filter, min_support=.01, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_itemset['product_cnt'] = frequent_itemset['itemsets'].apply(lambda x: len(x))
frequent_itemset

### Calculate the support, confidence, and lift values for each possible pair of products

In [14]:
from mlxtend.frequent_patterns import association_rules

product_association = association_rules(frequent_itemset, metric='confidence', min_threshold=.7).sort_values(['support','confidence'], ascending=[False,False]).reset_index(drop=True)
product_association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(red hanging heart t-light holder),(white hanging heart t-light holder),0.058784,0.177953,0.042455,0.722222,4.058510,0.031995,2.959371,0.800671
1,(sweetheart ceramic trinket box),(strawberry ceramic trinket box),0.049254,0.074980,0.037457,0.760487,10.142533,0.033764,3.862089,0.948103
2,(toilet metal sign),(bathroom metal sign),0.026993,0.040589,0.021728,0.804938,19.831353,0.020632,4.918499,0.975918
3,(red retrospot sugar jam bowl),(red retrospot small milk jug),0.023660,0.037123,0.016796,0.709859,19.121592,0.015917,3.318652,0.970669
4,(painted metal pears assorted),(assorted colour bird ornament),0.021927,0.078512,0.016596,0.756839,9.639738,0.014874,3.789618,0.916356
...,...,...,...,...,...,...,...,...,...,...
81,"(green 3 piece mini dots cutlery set, blue 3 p...",(pink 3 piece mini dots cutlery set),0.011464,0.029992,0.010064,0.877907,29.271370,0.009720,7.944827,0.977037
82,"(green 3 piece mini dots cutlery set, pink 3 p...",(blue 3 piece mini dots cutlery set),0.011997,0.030525,0.010064,0.838889,27.481853,0.009698,6.017430,0.975313
83,"(key fob , back door , key fob , front door )","(key fob , shed)",0.012463,0.025060,0.010064,0.807487,32.222153,0.009752,5.064272,0.981194
84,"(poppy's playhouse livingroom, poppy's playhou...",(poppy's playhouse bathroom),0.013730,0.010997,0.010064,0.733010,66.655016,0.009913,3.704266,0.998709


# Insight

1. The product Red Hanging Heart T-light Holder is purchased in 5.9% of total transactions.
2. When customers purchase the Red Hanging Heart T-light Holder, 72% of them also buy the White Hanging Heart T-light Holder.
3. There is a strong association between the Red Hanging Heart T-light Holder and the White Hanging Heart T-light Holder, with the likelihood of these two products being purchased together in one transaction being 4 times higher than the likelihood of each product being purchased individually.
4. Based on these insights, the White Hanging Heart T-light Holder can be placed near or next to the Red Hanging Heart T-light Holder, or both products can be sold as a bundle.